In [1]:
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr
import rioxarray
import scipy.io as sio
import datetime

Filepath list of .mat files

In [2]:
files = [r'C:\Users\steve\OneDrive\Documents\School Stuff\UW\Mountain Hydrology Research Group\IR_PLANE_PROCESSED-20200908T002540Z-001\IR_PLANE_PROCESSED\SNOWEX2020_IR_PLANE_2020Feb08_mosaicked.mat',
        r'C:\Users\steve\OneDrive\Documents\School Stuff\UW\Mountain Hydrology Research Group\IR_PLANE_PROCESSED-20200908T002540Z-001\IR_PLANE_PROCESSED\SNOWEX2020_IR_PLANE_2020Feb10_mosaicked.mat',
        r'C:\Users\steve\OneDrive\Documents\School Stuff\UW\Mountain Hydrology Research Group\IR_PLANE_PROCESSED-20200908T002540Z-001\IR_PLANE_PROCESSED\SNOWEX2020_IR_PLANE_2020Feb11_mosaicked.mat',
        r'C:\Users\steve\OneDrive\Documents\School Stuff\UW\Mountain Hydrology Research Group\IR_PLANE_PROCESSED-20200908T002540Z-001\IR_PLANE_PROCESSED\SNOWEX2020_IR_PLANE_2020Feb12_mosaicked.mat'
        ]

Load a .mat file from the filepath list.

In [3]:
aircraft_data_mat = sio.loadmat(files[0])  

Display the contents of what we have loaded. This is a dictionary containing metadata and numpy arrays of the data variables.

In [4]:
aircraft_data_mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Wed Aug 19 18:54:43 2020',
 '__version__': '1.0',
 '__globals__': [],
 'Eutm': array([[218150, 218155, 218160, ..., 238990, 238995, 239000]]),
 'Nutm': array([[4314100],
        [4314105],
        [4314110],
        ...,
        [4331990],
        [4331995],
        [4332000]]),
 'STCtemp': array([[[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan]],
 
        [[      nan,       nan,       nan, ...,       nan,       nan,
  

Create and xarray dataset with this data

In [5]:
# convert time in epoch to isoformat
time = [datetime.datetime.utcfromtimestamp(this_time).isoformat() for this_time in aircraft_data_mat['time'].squeeze()]


# create dataset
ds = xr.Dataset({
    'STCtemp': xr.DataArray(
                data   = aircraft_data_mat['STCtemp'], # scaled IR temperature data
                dims   = ['y', 'x', 'time'],
                coords = {'x': aircraft_data_mat['Eutm'].squeeze(), # easting coordinates (MGRS-UTM, m
                          'y': aircraft_data_mat['Nutm'].squeeze(), # northing grid coordinates (MGRS-UTM, 
                          'time': time}, # time in isoformat

                attrs  = {
                    '_FillValue'  : np.nan,
                    'units'       : 'Celsius',
                    'description' : 'land and snow surface brightness temperature - no emissivity correction',
                    'timezone'    : 'time in UTC'
                    }
                ),
    'zDEM': xr.DataArray(
                data   = aircraft_data_mat['zDEM'], # elevation (meters)
                dims   = ['y', 'x'],
                coords = {'x': aircraft_data_mat['Eutm'].squeeze(),
                          'y': aircraft_data_mat['Nutm'].squeeze()},
                attrs  = {
                    '_FillValue'  : np.nan,
                    'units'       : 'meters',
                    'description' : 'digital elevation map used for georectification',
                    'source'      : 'https://doi.org/10.5066/F7PR7TFT',
                    'horizontal datum' : 'WGS84',
                    'vertical datum' : 'EGM96',
                    'original resolution' : '30 m / 1 arc-second'
                    }
            )
},
        attrs = {'description': 'Airborne thermal infrared from University of Washington Applied Physics Lab Compact Airborne System for Imaging the Environment instrument. Imagery of Grand Mesa, Colorado from SnowEx 2020 field campaign.'}
    )



Specify our spatial dimensions and coordinate reference system

In [6]:
ds.rio.set_spatial_dims('y', 'x', inplace=True)
ds.rio.write_crs("epsg:26912", inplace=True) # UTM Zone 12N / NAD83

<xarray.Dataset>
Dimensions:      (time: 17, x: 4171, y: 3581)
Coordinates:
  * x            (x) int32 218150 218155 218160 218165 ... 238990 238995 239000
  * y            (y) int32 4314100 4314105 4314110 ... 4331990 4331995 4332000
  * time         (time) <U26 '2020-02-08T15:07:17.946189' ... '2020-02-08T19:56:16.283170'
    spatial_ref  int32 0
Data variables:
    STCtemp      (y, x, time) float32 nan nan nan nan nan ... nan nan nan nan
    zDEM         (y, x) float32 1935.4832 1937.1373 ... 2911.9229 2912.754
Attributes:
    description:   Airborne thermal infrared from University of Washington Ap...
    grid_mapping:  spatial_ref

In [7]:
ds.to_netcdf('SNOWEX2020_IR_PLANE_2020Feb08_mosaicked.nc')

For each timestamp, save a separate netcdf file.

In [8]:
for t, _ in enumerate(time):
    ds.isel(time=t).to_netcdf('SNOWEX2020_IR_PLANE_2020Feb08_mosaicked_{}.nc'.format(time[t].replace(':','').split('.')[0]))